<a href="https://colab.research.google.com/github/PabloAMC/fast_ai_practice/blob/main/Exercise_2_Yelp_polarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 2: text classification

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import libraries and so on

In [ ]:
! pip install fastai --upgrade
from fastai.text import *

Requirement already up-to-date: fastai in /usr/local/lib/python3.6/dist-packages (1.0.61)


## Preparing the data

In [ ]:
path = untar_data(URLs.YELP_REVIEWS_POLARITY); path

PosixPath('/root/.fastai/data/yelp_review_polarity_csv')

In [ ]:
path.ls()

[PosixPath('/root/.fastai/data/yelp_review_polarity_csv/readme.txt'),
 PosixPath('/root/.fastai/data/yelp_review_polarity_csv/data_save.pkl'),
 PosixPath('/root/.fastai/data/yelp_review_polarity_csv/train.csv'),
 PosixPath('/root/.fastai/data/yelp_review_polarity_csv/test.csv')]

In [ ]:
df = pd.read_csv(path/'train.csv', header= None)
df.head()

,0,1
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [ ]:
df[1][0]

"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."

We can create a dataset with a single line of code

In [ ]:
data_lm = TextDataBunch.from_csv(path, 'train.csv')

In [ ]:
data_lm.save()

In [ ]:
# Next time we can just do
data = load_data(path)

### Tokenization

In [ ]:
data = TextClasDataBunch.from_csv(path, 'train.csv')
data.show_batch()

/usr/local/lib/python3.6/dist-packages/fastai/text/data.py:339: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  idx_min = (t != self.pad_idx).nonzero().min()


text,target
xxbos a hands down must see attraction . xxmaj the first time i saw it in 2006 they played xxmaj my xxmaj heart xxmaj will xxmaj go xxmaj on by xxmaj celine xxmaj dion . xxmaj it was so beautiful i almost cried ! xxmaj the strip side offers the best views in my opinion . \n \n xxmaj the fountains are set in a 9 acre manmade lake,2
"xxbos xxmaj as it was "" xxmaj restaurant xxmaj week "" in xxmaj las xxmaj vegas , xxup rm xxmaj seafood offered a prix fixe for $ xxunk . xxmaj the second course of this prix fixe was the signature "" xxup rm xxmaj cioppino "" which normally runs around $ 33 by itself . xxmaj we had the prix fixe . \n \n xxmaj the host was professional",2
"xxbos i do n't really write reviews , but i just have to share the horrible experience i had at this hotel when i stayed this past weekend . i was excited for this weekend getaway to xxmaj vegas ! xxmaj who ever is n't ? xxmaj plus one of the girls found an awesome deal for this place on travelzoo for a one bedroom suite . xxmaj the pictures",1
"xxbos a while back a few of us went to see "" xxmaj the xxmaj heat "" , xxmaj about 10 - 15min after the previews should have started an usher comes in to let us know "" someone did n't do their job and had to transfer the the movie to this theater last minute "" the person did n't set up the movie properly causing it to not",1
"xxbos xxup warning xxup worst & xxup most xxup dangerous experience xxup ever . \n \n xxmaj since they opened our family / friends were regular patrons of this local drive thru / diner , which i 'm sorry to say is now more of a dive . xxmaj the former owners & staff were so nice that although the portions may not have always been great value for",1


### Numericalization

In [ ]:
data.vocab.itos[:10]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.']

In [ ]:
data.train_ds[0][0]

Text [   2    5  385 2151 ...   19   71   45    9]

### With the DataBlock API

In [ ]:
data.train_ds[0][0]

Text [   2    5  385 2151 ...   19   71   45    9]

In [ ]:
data = (TextList.from_csv(path, 'train.csv', cols = 1)
                .split_by_rand_pct(seed=1)
                .label_from_df(cols = 0)
                .databunch())

## Language model

Next we can create a language model without labels

In [ ]:
bs = 4

In [ ]:
data_lm = (TextList.from_csv(path, 'train.csv', cols = 1)
           #Inputs: all the text files in path
           #.filter_by_folder(include=['train', 'test']) 
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs))
data_lm.save('data_lm.pkl')

In [ ]:
data_lm.show_batch

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult = .3)

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_end=15)

In [ ]:
learn.fit_one_cycle(1, slice(9e-2,8e-1), moms = (0.8,0.7))

In [ ]:
learn.save('fit_head')

In [ ]:
learn.freeze_to(-2)

In [ ]:
learn.fit_one_cycle(1, slice(9e-2,8e-1), moms = (0.8,0.7))

In [ ]:
learn.save('fine_tuned')

In [ ]:
learn.load('fine_tuned')

Lets check for prediction

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
learn.save_encoder('fine_tuned_enc')

## Classifier

In [ ]:
data_clas = (TextList.from_csv(path, 'train.csv', cols = 1)
           #Inputs: all the text files in path
           #.filter_by_folder(include=['train', 'test']) 
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_from_df()           
           #We can label the data using the method from_df() because we are using from_csv or from_df
            .databunch(bs=bs))

data_clas.save('data_clas.pkl')

In [ ]:
data_clas = load_data(path, 'data_clas.pkl', bs=bs)

In [ ]:
data_clas.show_batch()

Create a model to clasify the data using the encoder 'fine_tuned_enc'

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

Save this model

In [ ]:
learn.save('first')
learn.load('first')

Unfreeze some of the previous layers

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.save('second')
learn.load('second');

Again, with one more layer

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.save('third')
learn.load('third')

Finally

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

Predict if a review is good or bad

In [ ]:
learn.predict("I really loved that movie, it was awesome!")